# SHN and Route Typology
* https://docs.google.com/spreadsheets/d/1gmRmVC4phwA3EunOhI4-aJ7uF5R2nZhIhPV2h3FM25w/edit?gid=0#gid=0

## Questions
* Is `route_typology` refreshed January of each year?
* Do I need to go back to 2023 and add back the route typologies? Or can I just add route typologies from August onward?
* Best way to troubleshoot why a dataframe increases in rows after a merge?
* What's the difference between `shape_id` in `open_data/create_routes` vs `common_shape_id` in `route_typology_df?`

## Steps
1. start with: open data routes (1 day? most recent date...for the most part, operator-service_date-route-shape). 
bring in route typology (which is a route's designation for that year) can be merged onto open data, needs an aggregation here to operator-route. this should be merged onto open data routes with a m:1 merge  (open data routes on left, route typology on right, merge on route)
2. to the above, you want to be able to tag those routes as being on shn or not. open data routes will have columns for is_on_shn, route_typology (is_express, is_rapid, is_rail, etc).
3. newer research task: the comparison of "what do you miss when you only sample Wed) is using merge_data.concatenate_schedule_data_by_route_direction(use a week here, instead of Wed), and you're doing this at some point, and this is when you start with gtfs_digest/merge_dataand this stuff is exploratory in a notebook

In [1]:
import geopandas as gpd
import google.auth
import numpy as np
import pandas as pd

credentials, project = google.auth.default()

import gcsfs

fs = gcsfs.GCSFileSystem()
import yaml

In [2]:
from calitp_data_analysis import geography_utils, utils
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from shared_utils import (
    catalog_utils,
    dask_utils,
    gtfs_utils_v2,
    portfolio_utils,
    publish_utils,
    rt_dates,
    rt_utils,
)
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [3]:
TRAFFIC_OPS_GCS = f"{GTFS_DATA_DICT.gcs_paths.GCS}traffic_ops/"

In [4]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [5]:
bart_org_name = "San Francisco Bay Area Rapid Transit District"
bart_gtfs_dataset_name = "Bay Area 511 BART Schedule"

In [6]:
analysis_date = rt_dates.DATES["jul2025"]

### Check results

In [7]:
og_url = "gs://calitp-analytics-data/data-analyses/traffic_ops/ca_transit_routes.parquet"

In [8]:
og_gdf = gpd.read_parquet(og_url,
    storage_options={"token": credentials.token},
)

In [9]:
test_url = "gs://calitp-analytics-data/data-analyses/ah_testing/ca_transit_routes.parquet"

In [10]:
test_gdf = gpd.read_parquet(test_url,
    storage_options={"token": credentials.token},
)

In [11]:
test_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 8101 entries, 0 to 14479
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   agency                             8101 non-null   object  
 1   route_id                           8101 non-null   object  
 2   route_type                         8101 non-null   object  
 3   route_name                         8101 non-null   object  
 4   route_length_feet                  8101 non-null   float64 
 5   shape_id                           8101 non-null   object  
 6   n_trips                            8101 non-null   int64   
 7   base64_url                         8101 non-null   object  
 8   shn_route                          6128 non-null   object  
 9   on_shs                             8101 non-null   int64   
 10  shn_districts                      6128 non-null   object  
 11  pct_route_on_hwy_across_districts 

In [12]:
test_gdf.drop(columns = ["geometry"]).sample()

,agency,route_id,route_type,route_name,route_length_feet,shape_id,n_trips,base64_url,shn_route,on_shs,shn_districts,pct_route_on_hwy_across_districts,is_express,is_ferry,is_rail,is_coverage,is_local,is_downtown_local,is_rapid
5534,Mountain Area Regional Transit Authority,240,3,2,107404.92,p_1439418,4,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3Rmcy9iaWdiZWFyLWNhLXVzL2JpZ2JlYXItY2EtdXMuemlw,"18, 189, 138, 173",1,8,55.90,0.00,0.00,0.00,1.00,1.00,0.00,0.00


In [13]:
set(list(test_gdf.columns)) - set(list(og_gdf.columns))

set()

* Insert route typology before organization name is merged in. 
* When I'm still working with feed key/gtfs schedule that's when I want to start adding things. 
* Move my merge to just name and route_id that will be a lot cleaner. 

In [ ]:
ca_transit_routes = gpd.read_parquet(
    f"gs://calitp-analytics-data/data-analyses/traffic_ops/ca_transit_routes.parquet",
    filters=[[("agency", "==", bart_org_name)]],
    storage_options={"token": credentials.token},
)

* Make sure the year is something I consider.
* Can concat all of the year to see which ones can merge on. 
* Because we patch previous dates in, we might be missing stuff if we only grab 2025. 
* After dataframe has been patched with `patch_previous_dates`, see what's left.
* `standardize_operator_info_for_exports` needs to be moved out and added on after I add in the SHN and route typologies. 
* All organization name renaming stuff needs to move to the end. 
* Read only what I need.

In [ ]:
year = "2025"
route_typologies = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/gtfs_schedule/"
    f"nacto_typologies/route_typologies_{year}.parquet",
    filters=[[("name", "==", bart_gtfs_dataset_name)]],
)

In [ ]:
ca_transit_routes.head(1).drop(columns=["geometry"])

In [ ]:
route_typologies.head(1)

* Always use `name` and `route_id`
* `route_typologies` is not always filled in for everythign we have.
* There are a number of reasons, Tiffany samples the dates and only runs it quarterly.
* Merging something daily to quarterly doesn't always work the best.
* Don't use organization_name. 
* Get rid of all the shape_id variations for intersecting.
    * WE just have to publish all of the `shape_id` because there are truly different variations.
    * 

Steps
1. Patch missing data
2. Merge `route_typologies` with `open_data` on the left
3. Check that I'm not missing too many.
4. Add in SHN stuff.
    * Can dedup `name, route_id,` keep one `shape_id` with the longest route length in a separate dataframe as the "representative" shape. 
    * Use that `shape_id` for doing the intersection between transit route x SHN to determine the column we are creating.
    * Merge this new dataframe back to the original dataframe. 
5. Add in `standardize_org_name` function
6. Add in `finalize_export_df` 

In [ ]:
test_m1 = pd.merge(
    ca_transit_routes.assign(name=bart_gtfs_dataset_name),
    route_typologies,
    on=["name", "route_id"],
    how="left",
    indicator=True,
)
test_m1._merge.value_counts()

In [ ]:
test_m1.loc[test_m1.route_id.isin(["Grey-N", "Grey-S"])].drop(
    columns=["geometry", "base64_url"]
)

In [ ]:
test_m1.loc[test_m1._merge == "left_only"].drop(columns=["geometry"])

In [ ]:
test_m2 = pd.merge(
    ca_transit_routes.assign(name=bart_gtfs_dataset_name),
    route_typologies,
    on=["name", "route_id", "route_type"],
    how="left",
    indicator=True,
)
test_m2._merge.value_counts()

In [ ]:
len(ca_transit_routes)

## Step 1: Concat all the years for `route_typologies`

In [ ]:
def add_route_typologies() -> pd.DataFrame:
    """
    Concatenate the years available for
    route typologies on the operator-route_id
    grain.
    """
    ROUTE_TYPOLOGIES_FILE = GTFS_DATA_DICT.schedule_tables.route_typologies

    route_typology_paths = [
        f"{SCHED_GCS}{ROUTE_TYPOLOGIES_FILE}" for year in rt_dates.years_available
    ]
    route_typology_df = dask_utils.get_ddf(
        route_typology_paths,
        rt_dates.years_available,
        data_type="df",
        get_pandas=True,
        columns=[
            "name",
            "route_id",
            "is_express",
            "is_ferry",
            "is_rail",
            "is_coverage",
            "is_local",
            "is_downtown_local",
            "is_rapid",
        ],
        add_date=False,
        add_year=True,
    )

    # Drop duplicates of operator-route_id to keep only the
    # row with the most current year.
    route_typology_df2 = route_typology_df.sort_values(
        by=["name", "route_id", "year"], ascending=[True, True, False]
    ).drop_duplicates(
        subset=[
            "name",
            "route_id",
        ]
    )
    return route_typology_df2

In [ ]:
ROUTE_TYPOLOGIES_FILE = GTFS_DATA_DICT.schedule_tables.route_typologies

In [ ]:
route_typology_df = add_route_typologies()

In [ ]:
route_typology_df.loc[route_typology_df.name == "Yolobus Schedule"].head(10)

In [ ]:
# route_typology_paths = [
#    f"{SCHED_GCS}{ROUTE_TYPOLOGIES_FILE}" for year in rt_dates.years_available
# ]

In [ ]:
# route_typology_df.head(2)

In [ ]:
# route_typology_df.sort_values(
#    by=["name", "route_id", "year"], ascending=[True, True, False]
# ).head()

In [ ]:
# route_typology_df2 = route_typology_df.sort_values(
#    by=["name", "route_id", "year"], ascending=[True, True, False]
# ).drop_duplicates(
#    subset=[
#        "name",
#        "route_id",
#    ]
# )

In [ ]:
# route_typology_df2.year.value_counts()

## Step 2: Patch missing data (remove `standardize org name`) function

In [ ]:
def remove_erroneous_shapes(
    shapes_with_route_info: gpd.GeoDataFrame,
) -> gpd.GeoDataFrame:
    """
    Check if line is simple for Amtrak. If it is, keep.
    If it's not simple (line crosses itself), drop.

    In Jun 2023, some Amtrak shapes appeared to be funky,
    but in prior months, it's been ok.
    Checking for length is fairly time-consuming.
    """
    amtrak = "Amtrak Schedule"

    possible_error = shapes_with_route_info[shapes_with_route_info.name == amtrak]
    ok = shapes_with_route_info[shapes_with_route_info.name != amtrak]

    # Check if the line crosses itself
    ok_amtrak = (
        possible_error.assign(simple=possible_error.geometry.is_simple)
        .query("simple == True")
        .drop(columns="simple")
    )

    ok_shapes = pd.concat([ok, ok_amtrak], axis=0).reset_index(drop=True)

    return ok_shapes

In [ ]:
def create_routes_file_for_export(date: str) -> gpd.GeoDataFrame:
    """
    Create a shapes (with associated route info) file for export.
    This allows users to plot the various shapes,
    transit path options, and select between variations for
    a given route.
    """
    # Read in local parquets
    trips = helpers.import_scheduled_trips(
        date,
        columns=[
            "name",
            "gtfs_dataset_key",
            "route_id",
            "route_type",
            "shape_id",
            "shape_array_key",
            "route_long_name",
            "route_short_name",
            "route_desc",
        ],
        get_pandas=True,
    ).dropna(subset="shape_array_key")

    shapes = helpers.import_scheduled_shapes(
        date,
        columns=["shape_array_key", "n_trips", "geometry"],
        get_pandas=True,
        crs=geography_utils.WGS84,
    ).dropna(subset="shape_array_key")

    df = (
        pd.merge(shapes, trips, on="shape_array_key", how="inner")
        .drop_duplicates(subset="shape_array_key")
        .drop(columns="shape_array_key")
    )

    drop_cols = ["route_short_name", "route_long_name", "route_desc"]
    route_shape_cols = ["schedule_gtfs_dataset_key", "route_id", "shape_id"]

    routes_assembled = (
        portfolio_utils.add_route_name(df)
        .drop(columns=drop_cols)
        .sort_values(route_shape_cols)
        .drop_duplicates(subset=route_shape_cols)
        .reset_index(drop=True)
    )

    routes_assembled = routes_assembled.pipe(remove_erroneous_shapes)

    routes_assembled = routes_assembled.assign(
        route_length_feet=routes_assembled.geometry.to_crs(
            geography_utils.CA_NAD83Albers_ft
        ).length
    )
    return routes_assembled

In [ ]:
routes = create_routes_file_for_export(analysis_date)

In [ ]:
def patch_previous_dates(
    current_routes: gpd.GeoDataFrame,
    current_date: str,
    published_operators_yaml: str = "../gtfs_funnel/published_operators.yml",
) -> gpd.GeoDataFrame:
    """
    Compare to the yaml for what operators we want, and
    patch in previous dates for the 10 or so operators
    that do not have data for this current date.
    """
    # Read in the published operators file
    with open(published_operators_yaml) as f:
        published_operators_dict = yaml.safe_load(f)

    # Convert the published operators file into a dict mapping dates to an iterable of operators
    patch_operators_dict = {
        str(date): operator_list
        for date, operator_list in published_operators_dict.items()
        if str(date)
        != current_date  # Exclude the current (analysis) date, since that does not need to be patched
    }

    partial_dfs = []

    # For each date and corresponding iterable of operators, get the data from the last time they appeared
    for one_date, operator_list in patch_operators_dict.items():
        df_to_add = publish_utils.subset_table_from_previous_date(
            gcs_bucket=TRAFFIC_OPS_GCS,
            filename=f"ca_transit_routes",
            operator_and_dates_dict=patch_operators_dict,
            date=one_date,
            crosswalk_col="schedule_gtfs_dataset_key",
            data_type="gdf",
        )

        partial_dfs.append(df_to_add)

    patch_routes = pd.concat(partial_dfs, axis=0, ignore_index=True)

    # Concat the current data to the "backfill" data
    published_routes = pd.concat(
        [current_routes, patch_routes], axis=0, ignore_index=True
    )

    # Drop Duplicates
    published_routes = published_routes.drop_duplicates().reset_index(drop=True)

    return published_routes

In [ ]:
routes.sample().drop(columns=["geometry"])

In [ ]:
len(routes)

In [ ]:
published_routes = patch_previous_dates(
    routes,
    analysis_date,
)

In [ ]:
len(published_routes)

In [ ]:
len(published_routes.drop_duplicates())

In [ ]:
len(published_routes.drop_duplicates(subset=["schedule_gtfs_dataset_key", "route_id"]))

In [ ]:
published_routes.info()

## Step 3: Merge `route_typologies` with `open_data`

In [ ]:
def add_route_typologies(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Concatenate the years available for
    route typologies on the operator-route_id
    grain.
    """
    ROUTE_TYPOLOGIES_FILE = GTFS_DATA_DICT.schedule_tables.route_typologies

    route_typology_paths = [
        f"{SCHED_GCS}{ROUTE_TYPOLOGIES_FILE}" for year in rt_dates.years_available
    ]
    route_typology_df = dask_utils.get_ddf(
        route_typology_paths,
        rt_dates.years_available,
        data_type="df",
        get_pandas=True,
        columns=[
            "name",
            "route_id",
            "is_express",
            "is_ferry",
            "is_rail",
            "is_coverage",
            "is_local",
            "is_downtown_local",
            "is_rapid",
        ],
        add_date=False,
        add_year=True,
    )

    # Drop duplicates of operator-route_id to keep only the
    # row with the most current year.
    route_typology_df2 = route_typology_df.sort_values(
        by=["name", "route_id", "year"], ascending=[True, True, False]
    ).drop_duplicates(
        subset=[
            "name",
            "route_id",
        ]
    )

    m1 = pd.merge(
        gdf,
        route_typology_df2,
        on=["name", "route_id"],
        how="left",
    )
    return m1

In [ ]:
m1 = add_route_typologies(published_routes)

In [ ]:
# m1 = pd.merge(
#    published_routes,
#   route_typology_df,
#   on=["name", "route_id"],
#  how="left",
# )

In [ ]:
len(m1)

In [ ]:
len(published_routes)

In [ ]:
len(published_routes.drop_duplicates())

In [ ]:
len(m1.drop_duplicates())

## Step 4: Add in SHN 
* <s>Keep only the `shape_id` with the longest length</s> joining on all the rows works pretty quickly.

In [ ]:
def routes_shn_intersection(
    routes_gdf: gpd.GeoDataFrame, buffer_amount: int
) -> gpd.GeoDataFrame:
    """
    Overlay the most recent transit routes with a buffered version
    of the SHN
    """
    GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/shared_data/"

    # Read in buffered shn here or re buffer if we don't have it available.
    HWY_FILE = f"{GCS_FILE_PATH}shn_buffered_{buffer_amount}_ft_shn_dissolved_by_ct_district_route.parquet"

    if fs.exists(HWY_FILE):
        shn_routes_gdf = gpd.read_parquet(
            HWY_FILE, storage_options={"token": credentials.token}
        )
    else:
        shn_routes_gdf = shared_data.buffer_shn(buffer_amount)

    # Process the most recent transit route geographies and ensure the
    # CRS matches the SHN routes' GDF so the overlay doesn't go wonky.
    routes_gdf = routes_gdf.to_crs(shn_routes_gdf.crs)

    # Overlay transit routes with the SHN geographies.
    gdf = gpd.overlay(
        routes_gdf, shn_routes_gdf, how="intersection", keep_geom_type=True
    )

    # Calcuate the percent of the transit route that runs on a highway, round it up and
    # multiply it by 100. Drop the geometry because we want the original transit route
    # shapes.
    gdf = gdf.assign(
        pct_route_on_hwy=(gdf.geometry.length / gdf.route_length_feet).round(3) * 100,
    )
    # Subset
    gdf2 = gdf[
        [
            "name",
            "pct_route_on_hwy",
            "route_id",
            "shape_id",  # maybe comment out later
            "district",
            "shn_route",
        ]
    ]

    # Clean up
    gdf2.district = gdf2.district.fillna(0).astype(int)

    gdf2 = gdf2.rename(
        columns={
            "pct_route_on_hwy": "pct_route_on_hwy_across_districts",
            "district": "shn_districts",
        }
    )
    return gdf2

In [ ]:
# shn_routes = routes_shn_intersection(m1, 50)

In [ ]:
# shn_routes.sample(5)

In [ ]:
# shn_routes.pct_route_on_hwy_across_districts.describe()

In [ ]:
def group_route_district(df: pd.DataFrame, pct_route_on_hwy_agg: str) -> pd.DataFrame:
    """
    Aggregate by adding all the districts and SHN to a single row, rather than
    multiple and sum up the total % of SHN a transit route intersects with.

    df: the dataframe you want to aggregate
    pct_route_on_hwy_agg: whether you want to find the max, min, sum, etc on the column
    "pct_route_on_hwy_across_districts"
    """

    agg1 = (
        df.groupby(
            ["name", "route_id", "shape_id"],  # maybe comment out later
            as_index=False,
        )[["shn_route", "shn_districts", "pct_route_on_hwy_across_districts"]]
        .agg(
            {
                "shn_route": lambda x: ", ".join(set(x.astype(str))),
                "shn_districts": lambda x: ", ".join(set(x.astype(str))),
                "pct_route_on_hwy_across_districts": pct_route_on_hwy_agg,
            }
        )
        .reset_index(drop=True)
    )

    # Clean up
    agg1.pct_route_on_hwy_across_districts = (
        agg1.pct_route_on_hwy_across_districts.astype(float).round(2)
    )

    return agg1

In [ ]:
# grouped = group_route_district(shn_routes, "sum")

In [ ]:
# grouped.sample(10)

In [ ]:
# grouped.pct_route_on_hwy_across_districts.describe()

In [ ]:
# m1.columns

In [ ]:
# grouped.columns

In [ ]:
# len(grouped)

In [ ]:
# Merge back with the original dataframe
# shn_typology = pd.merge(m1, grouped, on=["route_id", "name", "shape_id"], how="left")

In [ ]:
# len(shn_typology), len(m1)

In [ ]:
def add_shn_information(gdf: gpd.GeoDataFrame, buffer_amt: int) -> pd.DataFrame:
    """
    Prepare the gdf to join with the existing transit_routes
    dataframe that is published on the Open Data Portal
    """
    # Retain only the longest shape for each name-route_id combo
    # so finding the intersection with SHN won't take as long
    """
    gdf = gdf.sort_values(
        by=["name", "route_id", "route_length_feet"], ascending=[True, True, False]
    )[["name", "route_id", "route_length_feet", "geometry"]].drop_duplicates(
        subset=["name", "route_id"]
    )
    """
    # Overlay
    intersecting = routes_shn_intersection(gdf, buffer_amt)

    # Group the dataframe so that one route only has one
    # row instead of multiple rows after finding its
    # intersection with any SHN routes.
    # print(intersecting.columns)
    agg1 = group_route_district(intersecting, "sum")

    # Merge the dataframe with all the SHS info with the original
    # gdf so we can get the original transit route geometries &
    # any routes that don't intersect with the state highway routes.
    m1 = pd.merge(gdf, agg1, on=["route_id", "name", "shape_id"], how="left")

    # Add yes/no column to signify if a transit route intersects
    # with a SHN route
    m1.pct_route_on_hwy_across_districts = m1.pct_route_on_hwy_across_districts.fillna(
        0
    )
    m1["on_shs"] = np.where(m1["pct_route_on_hwy_across_districts"] == 0, 0, 1)

    # Clean up rows that are tagged as "on_shs==N" but still have values
    # that appear.
    m1.loc[
        (m1["on_shs"] == "N") & (m1["shn_districts"] != "0"),
        ["shn_districts", "shn_route"],
    ] = np.nan

    return m1

In [ ]:
shn_typology = add_shn_information(m1, 50)

In [ ]:
type(shn_typology)

In [ ]:
len(shn_typology)

In [ ]:
len(m1)

In [ ]:
shn_typology.on_shs.value_counts()

In [ ]:
shn_typology.loc[
    (shn_typology.name == "Antelope Valley Transit Authority Schedule")
    & (shn_typology.route_id == "50")
].drop(columns=["geometry", "schedule_gtfs_dataset_key"]).T

In [ ]:
shn_typology.drop(columns=["geometry", "schedule_gtfs_dataset_key"]).sample(3).T

In [ ]:
shn_typology.info()

## Step 5: Edit `standardize_org_name` to be on `portfolio_organization_grain`

In [ ]:
def standardize_operator_info_for_exports(df: pd.DataFrame, date: str) -> pd.DataFrame:
    """
    Use our crosswalk file created in gtfs_funnel
    and add in the organization columns we want to
    publish on.
    """

    CROSSWALK_FILE = GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk

    public_feeds = gtfs_utils_v2.filter_to_public_schedule_gtfs_dataset_keys()

    # Get the crosswalk file
    crosswalk = pd.read_parquet(
        f"{SCHED_GCS}{CROSSWALK_FILE}_{date}.parquet",
        columns=[
            "schedule_gtfs_dataset_key",
            "name",
            "base64_url",
            "caltrans_district",
        ],
        filters=[[("schedule_gtfs_dataset_key", "in", public_feeds)]],
    )

    # Add portfolio_organization_name
    crosswalk = (
        crosswalk.assign(
            caltrans_district=crosswalk.caltrans_district.map(
                portfolio_utils.CALTRANS_DISTRICT_DICT
            )
        )
        .pipe(
            portfolio_utils.standardize_portfolio_organization_names,
            PORTFOLIO_ORGANIZATIONS_DICT,
        )
        .drop_duplicates(
            subset=["schedule_gtfs_dataset_key", "name", "portfolio_organization_name"]
        )
    )

    # Checked whether we need a left merge to keep stops outside of CA
    # that may not have caltrans_district
    # and inner merge is fine. All operators are assigned a caltrans_district
    # so Amtrak / FlixBus stops have values populated

    # Merge the crosswalk and the input DF
    crosswalk_input_merged = pd.merge(
        df,
        crosswalk,
        on=["schedule_gtfs_dataset_key"],
        suffixes=[
            "_original",
            None,
        ],  # Keep the source record id from the crosswalk as the "definitive" version
        how="inner",
    )

    # Drop dups
    crosswalk_input_merged = crosswalk_input_merged.drop_duplicates()
    return crosswalk_input_merged

In [ ]:
CROSSWALK_FILE = GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk

public_feeds = gtfs_utils_v2.filter_to_public_schedule_gtfs_dataset_keys()

In [ ]:
len(public_feeds)

In [ ]:
# Get the crosswalk file
crosswalk = pd.read_parquet(
    f"{SCHED_GCS}{CROSSWALK_FILE}_{analysis_date}.parquet",
    columns=[
        "schedule_gtfs_dataset_key",
        "name",
        "base64_url",
        "organization_source_record_id",
        "organization_name",
        "caltrans_district",
    ],
    filters=[[("schedule_gtfs_dataset_key", "in", public_feeds)]],
)

In [ ]:
crosswalk.shape

In [ ]:
with open("../_shared_utils/shared_utils/portfolio_organization_name.yml", "r") as f:
    PORTFOLIO_ORGANIZATIONS_DICT = yaml.safe_load(f)

In [ ]:
crosswalk = crosswalk.assign(
    caltrans_district=crosswalk.caltrans_district.map(
        portfolio_utils.CALTRANS_DISTRICT_DICT
    )
).pipe(
    portfolio_utils.standardize_portfolio_organization_names,
    PORTFOLIO_ORGANIZATIONS_DICT,
)

In [ ]:
crosswalk.sort_values(by=["caltrans_district", "portfolio_organization_name"]).drop(
   columns=["base64_url"]
)

In [ ]:
len(crosswalk)

In [ ]:
len(crosswalk.drop_duplicates())

In [ ]:
crosswalk.columns

In [ ]:
len(crosswalk.drop_duplicates(subset=["portfolio_organization_name"]))

In [ ]:
len(crosswalk.drop_duplicates(subset=["schedule_gtfs_dataset_key", "name"]))

In [ ]:
len(
    crosswalk.drop_duplicates(
        subset=["schedule_gtfs_dataset_key", "name", "portfolio_organization_name"]
    )
)

In [ ]:
len(crosswalk.drop_duplicates(subset=["name", "portfolio_organization_name"]))

In [ ]:
shn_typology2 = standardize_operator_info_for_exports(shn_typology, analysis_date)

In [ ]:
len(shn_typology2), len(shn_typology)

## Step 6: Add in `finalize_export_df`

In [ ]:
shn_typology2.columns

In [ ]:
STANDARDIZED_COLUMNS_DICT = {
    "caltrans_district": "district_name",
    "organization_source_record_id": "org_id",
    "organization_name": "agency",
    "agency_name_primary": "agency_primary",
    "agency_name_secondary": "agency_secondary",
    "route_name_used": "route_name",
    "route_types_served": "routetypes",
    "meters_to_shn": "meters_to_ca_state_highway",
    "portfolio_organization_name": "agency",
}

In [ ]:
def finalize_export_df(df: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Suppress certain columns used in our internal modeling for export.
    """
    # Change column order
    route_cols = [
        "portfolio_organization_name",
        "route_id",
        "route_type",
        "route_name_used",
        "route_length_feet",
    ]
    shape_cols = ["shape_id", "n_trips"]
    agency_ids = ["base64_url"]
    shn_cols = [
        "shn_route",
        "on_shs",
        "shn_districts",
        "pct_route_on_hwy_across_districts",
    ]

    route_typology = [
        "is_express",
        "is_ferry",
        "is_rail",
        "is_coverage",
        "is_local",
        "is_downtown_local",
        "is_rapid",
    ]
    col_order = (
        route_cols + shape_cols + agency_ids + shn_cols + route_typology + ["geometry"]
    )

    df2 = (
        df[col_order]
        .reindex(columns=col_order)
        .rename(columns=STANDARDIZED_COLUMNS_DICT)
        .reset_index(drop=True)
    )

    return df2

In [ ]:
shn_typology2.columns

In [ ]:
shn_typology3 = shn_typology2.pipe(finalize_export_df)

## Check #1: There are many more rows after piping?

In [ ]:
shn_typology3.shape

In [ ]:
shn_typology3.info()

In [ ]:
shn_typology3.drop(columns=["geometry", "base64_url"]).sample(3)

In [ ]:
shn_typology3.pct_route_on_hwy_across_districts.describe()

In [ ]:
shn_typology3.agency.nunique()

In [ ]:
shn_typology2.shape

In [ ]:
len(shn_typology2.drop_duplicates())

In [ ]:
shn_typology.shape

In [ ]:
shn_typology.route_id.nunique()

In [ ]:
shn_typology3.route_id.nunique()

### Check original dataframes and see if this is the case.

In [ ]:
published_transit_routes = gpd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/traffic_ops/ca_transit_routes.parquet",
    storage_options={"token": credentials.token},
)

In [ ]:
july_16_routes = gpd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/traffic_ops/ca_transit_routes_2025-07-16.parquet",
    storage_options={"token": credentials.token},
)

In [ ]:
len(published_transit_routes) - len(july_16_routes)

In [ ]:
len(published_transit_routes)

In [ ]:
len(published_transit_routes.drop_duplicates())

In [ ]:
len(july_16_routes.drop_duplicates())

In [ ]:
july_16_routes.info()

## Check 2: Make sure `create_routes_data` works with my newly added lines
* Missing a lot of rows.

In [ ]:
ah_ca_transit_routes = gpd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/ah_testing/ca_transit_routes.parquet",
    storage_options={"token": credentials.token},
)

In [ ]:
ah_ca_transit_routes.info()

In [ ]:
ah_ca_july_routes = gpd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/ah_testing/ca_transit_routes_2025-07-16.parquet",
    storage_options={"token": credentials.token},
)

In [ ]:
ah_ca_july_routes.shape

In [ ]:
july_16_routes.shape

In [ ]:
ah_ca_transit_routes.shape

In [ ]:
ah_ca_transit_routes.info()

In [ ]:
shn_typology3.shape

In [ ]:
shn_typology3.info()

## Tiffany Code

In [ ]:
analysis_date = rt_dates.DATES["jul2025"]

public_feeds = gtfs_utils_v2.filter_to_public_schedule_gtfs_dataset_keys()
print(f"# of public schedule_gtfs_dataset_keys {len(public_feeds)}")

CROSSWALK_FILE = GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk

crosswalk = pd.read_parquet(
    f"{SCHED_GCS}{CROSSWALK_FILE}_{analysis_date}.parquet",
    columns=[
        "schedule_gtfs_dataset_key",
        "name",
        "base64_url",
        "organization_source_record_id",
        "organization_name",
        "caltrans_district",
    ],
    # filters = [[("schedule_gtfs_dataset_key", "in", public_feeds)]]
)


crosswalk2 = pd.read_parquet(
    f"{SCHED_GCS}{CROSSWALK_FILE}_{analysis_date}.parquet",
    columns=[
        "schedule_gtfs_dataset_key",
        "name",
        "base64_url",
        "organization_source_record_id",
        "organization_name",
        "caltrans_district",
    ],
    filters=[[("schedule_gtfs_dataset_key", "in", public_feeds)]],
)
print(f"no filter: {crosswalk.shape}")

print(f"filtered: {crosswalk2.shape}")